# **Homework 6 Solutions**

Due:

## **Written Assignment**

### **AdaBoost and Decision Stumps**

Below is a graph of points, which are classified as `+` or `-`. The
features are the $x$ and $y$ coordinates.

![image](hw6_02.png)

*Note:* Decision stumps are 1D decision trees. For continuous features
(in our case, x and y values) a threshold value is chosen, and there is
a chosen classification for inputs falling above or below that
threshold. For example, one decision stump may be the vertical line
represented `x `$>$` 2`, with values satisfying this (i.e., to the right
of the line) being classified one way, and values not satisfying this
(i.e., to the left of the line) being classified separately. Note that
decision stumps are **not** equivalent to halfspaces. See example
below.

![image](decision_stump.png)

### **Question 1**
Using decision stumps, produce images representing a simulation of
the Adaboost algorithm on this dataset. Represent the changing
relative weight of each datapoint by enlarging or shrinking the
symbol (+/-). Each image should represent one step of the algorithm,
and the last image should be the final classification of the
ensemble. Recall that decision stumps can only be vertical or
horizontal.

**Solution:**

![image](hw6_02_solution.png)

### **Question 2**
What is the difference between weak learners and strong learners in terms of their guarantees on the bounds on the error?

**Solution:**

Note these are complete definitions. The important aspects in the
student answers are the differences between the loss functions, in
particular the bounds. They should say something about how strong
learners can have arbitrarily small error while weak learners can
have a constant bound that is slightly better than random guessing.

**Weak Learners:** A class $C$ is weakly PAC learnable using a
hypothesis class $\mathcal{H}$ if there exists an algorithm $A$ and
a value $\gamma > 0$ such that for any $c \in C$, for any
distribution $\mathcal{D}$ over the input space, for any
$\delta \in (0, 1/2)$, given access to a polynomial (in $1/\delta$)
number of examples drawn i.i.d. from $\mathcal{D}$ and labeled by
$f$, $A$ outputs a function $h \in \mathcal{H}$ such that with
probability at least
$1-\delta, L_{(\mathcal{D},f)}(h) \leq \frac{1}{2} - \gamma$

**Strong Learners:** A class $C$ is strongly PAC learnable using a
hypothesis class $\mathcal{H}$ if there exists an algorithm $A$ such
that for any $c \in C$, for any distribution $\mathcal{D}$ over the
input space, for any $\epsilon \in (0, 1/2)$ and
$\delta \in (0, 1/2)$, given access to a polynomial (in $1/\epsilon$
and $1/\delta$) number of examples drawn i.i.d. from $\mathcal{D}$
and labeled by $f$, $A$ outputs a function $h \in \mathcal{H}$ such
that with probability at least
$1 -\delta, L_{(\mathcal{D},f)}(h) \leq \epsilon$.

### **Question 3**
What are the bounds on error of Adaboost at `T=1`? Remember that
`T=1` is the case when you have only one learner. What about as `T`
approaches infinity? How does this relate to the idea of strong and
weak learners?

**Solution:**

A weak learner by definition produces a loss better than random as
shown above. In AdaBoost, the training error is at most
$L_S(h_s) \leq \exp(-2\gamma ^2 T)$. At $T=1$ we effectively have a
decision stump, and as a result have a weak learner. However, as $T$
approaches infinity, even if $\gamma>0$ is arbitrarily small, we
approach a training error of 0. What this shows is that through the
combination of weak learners, AdaBoost is able generate a strong
learner.

# **Programming Assignment Solutions**

### Introduction

In this assignment, you will be implementing Decision Trees to solve
binary classification problems. By the end of this assignment, you will
have a classifier that you will use to predict the result of chess
matches and classify emails as spam. We recommend that you start this
assignment early, as the logic you have to implement may be complicated.

### Stencil Code

We have provided the following stencil code within this file:

-   `Models` contains the `DecisionTree` class, which will contain
    the bulk of the code you write.

-   `Check Model` contains a series of tests to ensure you are coding your 
    model properly.

-   `Main` is the entry point of your program, which will read in the
    data, run the classifiers and print the results. You will make small
    modifications to this file.

-   `Get Data` contains the data loading and processing. You do
    **not** need to change this file.


You should *not* modify any code in `Get Data`. All the functions you
need to fill in reside in `Main` and `Models`, marked by `TODO`s.

### Data

#### Spambase Dataset

You will be testing your Decision Trees on a real world dataset, the
Spambase dataset. Our goal is to train a model that can classify whether
an email is spam or not. The dataset features attributes such as the
frequency of certain words and the amount of capital letters in a given
message. You can find more details on the dataset
[**here**](https://archive.ics.uci.edu/ml/datasets/spambase). We will
only be using a subset of the full dataset.

#### Chess Dataset

Each row of the `chess.csv` dataset contains 36 features, which
represent the current state of the chess board. Given this
representation, the task is to use the Decision Trees to classify
whether or not it is possible for white to win the game. For more
information on the dataset, see
[**here**](https://archive.ics.uci.edu/ml/datasets/Chess+(King-Rook+vs.+King-Pawn)).

We have taken care of all the data preprocessing required so that you
can focus on implementing the machine-learning algorithms! We hope that
from these two examples, you can understand the versatility and power of
your abstract decision tree classifier.

## **The Assignment**
### **Decision Trees**

### Part I: Generic Decision Trees in Python

In this part, you will be implementing a generic decision tree for
binary classification given binary features. Your decision tree will
take training data
$S = ((\mathbf{x}_1, y_1) \dots (\mathbf{x}_m, y_m))$---where
$\mathbf{x}_{i} \in \{0, 1\}^{d}$ represents the binary feature vectors
and $y \in \{0, 1\}$ are the class labels---and attempt to find a tree
that minimizes training error. Recall that the training error for a
hypothesis $h$ is defined as the *average* $0\!-\!1$ loss

$$L_{S}(h) = \frac{1}{m} \sum\limits_{(\mathbf{x},y) \in S} (y \neq h(\mathbf{x})).$$

The primary methods of the `DecisionTree` class are as follows:

-   **Functionality:**

    -   `DecisionTree(data, validation_data=None, gain_function=node_score_entropy, max_depth=40)`
        creates a `DecisionTree` that greedily minimizes training error
        on the given dataset. The depth of the tree should not be
        greater than `max_depth`. If $\texttt{validation\_data}$ is
        passed as an argument, the validation data should be used to
        prune the tree after it has been constructed.

    -   `predict(features)` predicts a label $y \in \{0, 1\}$ given
        $\text{features} \in \{0,1\}^{d}$. Note that in our
        implementation features are represented as Python `bool` types
        (`True`, `False`) and class labels are Python `int`s (`0, 1`).

    -   `accuracy(data)` computes accuracy, defined as
        $1 - \texttt{loss(self, data)}$.

    -   `loss(data)` computes the training error, or the *average* loss,
        $L_{\texttt{data}}(h)$.

-   **Helper functions:** This is where most of the algorithmic work
    will take place. Each helper function begins with an underscore:
    `_predict_recurs, _prune_recurs, _is_terminal, _split_recurs, _calc_gain`.
    We already implemented `_predict_recurs` for you, so please do not
    modify that function. You should implement the other helper
    functions without changing the function signatures. **Note that when
    splitting on the $i$-th feature, the left child will have data
    points with $i$-th feature 0 and the right child will have data
    points with $i$-th feature 1.**

-   **Debugging:** We have given you two functions to help you visualize
    your decision tree for debugging purposes. You are free to use (or
    not use) them. We will not be grading you on whether you use or
    modify this code.

    -   `print_tree()` prints the tree to the command line. We have
        provided a working implementation, which you are free to
        improve. The current tree visualization works best for very
        shallow trees.

    -   `loss_plot_vec(data)` returns a vector of loss values where the
        $i$-th element corresponds to the loss of the tree with $i$
        nodes. The result can be plotted with `matplotlib.pyplot` to
        visualize the loss as your tree expands.

    To use these debugging functions, you must store the maximum gain at
    each node and the number of data points that have made it to that
    node when training using \_set_info().

Your task is to ensure that the `DecisionTree` class is fully
implemented. If you are unsure where to begin, we have provided `TODO`
comments in the stencil code to help get you started! We recommend
testing your code incrementally. It would be easiest to program
`_is_terminal, _calc_gain` and one of the gain functions first as they
are all needed in `_split_recurs`. You should start working on pruning
at the last step when you are sure that other functions work. You are
free to write your own tests for any of the provided functions to ensure
that they are working correctly.

### Part II: Measures of Gain

As mentioned in lecture, there are multiple measures of gain that an
algorithm can use when determining on which feature to split the current
node. In this assignment, you will be implementing and comparing the
results of three measures of gain: decrease in training error,
information gain (entropy) and Gini index. We recommend reviewing the
lecture slides or textbook if these terms sound unfamiliar.\
\
The `DecisionTree` class takes an optional `gain_function` parameter.
This function will be one of the three functions left for you to
implement: `node_score_error`, `node_score_entropy` and
`node_score_gini`. All of these gain functions should return a float.

### Part III: Chess Predictions & Spam Classification

Once you have implemented the `DecisionTree` class, you are ready to
explore the chess and spam datasets! You should now write code in
`main` that will print the following loss values:

-   For each dataset (`chess.csv`, `spam.csv`)

    -   For each gain function (Training error, Entropy, Gini)

        -   Print training loss without pruning

        -   Print test loss without pruning

        -   Print training loss with pruning

        -   Print test loss with pruning

Your final program should print **exactly** 24 lines of output. Each
line may contain text, but should end with the loss values defined
above.

In [27]:
#[TODO] run this cell to make sure you are in the right environment. 
# We will deduct 2 points for each missing OK sign.
from __future__ import print_function
from packaging.version import parse as Version
from platform import python_version

OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "\x1b[41m[FAIL]\x1b[0m"

try:
    import importlib
except ImportError:
    print(FAIL, "Python version 3.10 is required,"
                " but %s is installed." % sys.version)

def import_version(pkg, min_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        if pkg in {'PIL'}:
            ver = mod.VERSION
        else:
            ver = mod.__version__
        if Version(ver) == Version(min_ver):
            print(OK, "%s version %s is installed."
                  % (lib, min_ver))
        else:
            print(FAIL, "%s version %s is required, but %s installed."
                  % (lib, min_ver, ver))    
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return mod


# first check the python version
pyversion = Version(python_version())

if pyversion >= Version("3.10.7"):
    print(OK, "Python version is %s" % pyversion)
elif pyversion < Version("3.10.7"):
    print(FAIL, "Python version 3.10.7 is required,"
                " but %s is installed." % pyversion)
else:
    print(FAIL, "Unknown Python version: %s" % pyversion)

    
print()
requirements = {'matplotlib': "3.7.2", 'numpy': "1.24.4",'sklearn': "1.3.0", 
                'pandas': "2.0.3", "pytest": "7.2.1"}

# now the dependencies
for lib, required_version in list(requirements.items()):
    import_version(lib, required_version)

[ OK ] Python version is 3.10.7

[ OK ] matplotlib version 3.6.0 is installed.
[ OK ] numpy version 1.23.3 is installed.
[ OK ] sklearn version 1.1.1 is installed.
[ OK ] pandas version 1.4.2 is installed.


## **Get Data**

In [28]:
import random
import csv
import numpy as np

#####################################################################################################################
# Data Processing Section
# Helper function for preparing data for a decision tree classifiction problem. Parsing the data such
# that for each feature, the property can only either be True or False. Label can only be 1 or 0.
# For the chess.csv dataset won=1, nowin=0
# In more detail:
# Dataset with n instances, for each instance, there are m attributes. For the i-th attribute,
# the property should be chosen from a set with size of m_i to represent the information.
# Input: array with size of n*(m+1), the first column is the label
# Output: array with size of n*(m_1 + m_2 + ... + m_m + 1), the first column is 1 or 0 corresponding to label
#####################################################################################################################

def get_data(filename, class_name):
    data = read_data(filename)
    data = convert_to_binary_features(data, class_name)
    return np.array(split_data(data), dtype=object)

def read_data(filename):
    data = []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(row)
    return data

def convert_to_binary_features(data, class_name):
    features = []
    for feature_index in range(0, len(data[0])-1):
        feature_values = list(set([obs[feature_index] for obs in data]))
        feature_values.sort()
        if len(feature_values) > 2: features.append(feature_values[:-1])
        else: features.append([feature_values[0]])
    new_data = []
    for obs in data:
        new_obs = [1 if obs[-1] == class_name else 0] # label = 1 if label in the dataset is won
        for feature_index in range(0, len(data[0]) - 1):
            current_feature_value = obs[feature_index]
            for possible_feature_value in features[feature_index]:
                new_obs.append(current_feature_value == possible_feature_value)
        new_data.append(new_obs)

    return new_data

def split_data(data, num_training=1000, num_validation=1000):
    random.shuffle(data)
    # casting to a numpy array
    data = np.array(data)
    return data[0:num_training], data[num_training:num_training + num_validation], data[num_training + num_validation:len(data)]

## **Model**

In [29]:
import copy
import math

def node_score_error(prob):
    '''
        TODO:
        Calculate the node score using the train error of the subdataset and return it.
        For a dataset with two classes, C(p) = min{p, 1-p}
    '''
    error = min(prob, 1-prob)
    return error


def node_score_entropy(prob):
    '''
        TODO:
        Calculate the node score using the entropy of the subdataset and return it.
        For a dataset with 2 classes, C(p) = -p * log(p) - (1-p) * log(1-p)
        For the purposes of this calculation, assume 0*log0 = 0.
        HINT: remember to consider the range of values that p can take!
    '''
    # HINT: If p < 0 or p > 1 then entropy = 0
    if prob < 1 and prob > 0:
        entropy = -prob * math.log(prob) - (1-prob) * math.log(1-prob)
    else: 
        entropy = 0
    return entropy


def node_score_gini(prob):
    '''
        TODO:
        Calculate the node score using the gini index of the subdataset and return it.
        For dataset with 2 classes, C(p) = 2 * p * (1-p)
    '''
    return 2 * prob * (1-prob)


class Node:
    '''
    Helper to construct the tree structure.
    '''
    def __init__(self, left=None, right=None, depth=0, index_split_on=0, isleaf=False, label=1):
        self.left = left
        self.right = right
        self.depth = depth
        self.index_split_on = index_split_on
        self.isleaf = isleaf
        self.label = label
        self.info = {} # used for visualization


    def _set_info(self, gain, num_samples):
        '''
        Helper function to add to info attribute.
        You do not need to modify this. 
        '''

        self.info['gain'] = gain
        self.info['num_samples'] = num_samples


class DecisionTree:

    def __init__(self, data, validation_data=None, gain_function=node_score_entropy, max_depth=40):
        self.max_depth = max_depth
        self.root = Node()
        self.gain_function = gain_function

        indices = list(range(1, len(data[0])))

        self._split_recurs(self.root, data, indices)

        # Pruning
        if validation_data is not None:
            self._prune_recurs(self.root, validation_data)


    def predict(self, features):
        '''
        Helper function to predict the label given a row of features.
        You do not need to modify this.
        '''
        return self._predict_recurs(self.root, features)


    def accuracy(self, data):
        '''
        Helper function to calculate the accuracy on the given data.
        You do not need to modify this.
        '''
        return 1 - self.loss(data)


    def loss(self, data):
        '''
        Helper function to calculate the loss on the given data.
        You do not need to modify this.
        '''
        cnt = 0.0
        test_Y = [row[0] for row in data]
        for i in range(len(data)):
            prediction = self.predict(data[i])
            if (prediction != test_Y[i]):
                cnt += 1.0
        return cnt/len(data)


    def _predict_recurs(self, node, row):
        '''
        Helper function to predict the label given a row of features.
        Traverse the tree until leaves to get the label.
        You do not need to modify this.
        '''
        if node.isleaf or node.index_split_on == 0:
            return node.label
        split_index = node.index_split_on
        if not row[split_index]:
            return self._predict_recurs(node.left, row)
        else:
            return self._predict_recurs(node.right, row)


    def _prune_recurs(self, node, validation_data):
        '''
        TODO:
        Prune the tree bottom up recursively. Nothing needs to be returned.
        Do not prune if the node is a leaf.
        Do not prune if the node is non-leaf and has at least one non-leaf child.
        Prune if deleting the node could reduce loss on the validation data.
        NOTE:
        This might be slightly different from the pruning described in lecture.
        Here we won't consider pruning a node's parent if we don't prune the node 
        itself (i.e. we will only prune nodes that have two leaves as children.)
        HINT: Think about what variables need to be set when pruning a node!
        '''
        # Checks if node is not a Leaf
        if not node.isleaf:
            if node.left is not None:
                # TODO: Prune node.left
                self._prune_recurs(node.left, validation_data)
            if node.right is not None:
                # TODO: Prune node.right
                self._prune_recurs(node.right, validation_data)
            if (node.left.isleaf) and (node.right.isleaf):
                # TODO: Prune node only if loss is reduced
                old_loss = self.loss(validation_data)
                left_node = node.left
                right_node = node.right
                node.left = None
                node.right = None
                node.isleaf = True 
                new_loss = self.loss(validation_data)

                if new_loss > old_loss:
                    node.isleaf = False
                    node.left = left_node
                    node.right = right_node
                    
#             # shorter method for the last if statement
#             if (node.left.isleaf) and (node.right.isleaf):
#                 # TODO: Prune node only if loss is reduced
#                 old_loss = self.loss(validation_data)
#                 node.isleaf = True 
#                 new_loss = self.loss(validation_data)

#                 if new_loss > old_loss:
#                     node.isleaf = False
                    
        
    def _is_terminal(self, node, data, indices):
        '''
        TODO:
        Helper function to determine whether the node should stop splitting.
        Stop the recursion if:
            1. The dataset is empty.
            2. There are no more indices to split on.
            3. All the instances in this dataset belong to the same class
            4. The depth of the node reaches the maximum depth.
        Return:
            - A boolean, True indicating the current node should be a leaf and 
              False if the node is not a leaf.
            - A label, indicating the label of the leaf (or the label the node would 
              be if we were to terminate at that node). If there is no data left, you
              can return either label at random.
        '''
        y = [row[0] for row in data]
        isleaf = False
        label = 0

        if len(y) == 0: 
            # TODO: If there is no more data return random label
            isleaf = True
            label = random.randint(2)
        else:
            # TODO: Check Cases if the node should stop splitting and update label
            condition = (len(indices)==0) or \
            (np.mean(y)==0) or (np.mean(y)==1) or \
            (node.depth == self.max_depth)
            
            if condition == True:
                isleaf = True
            else:
                isleaf = False
            
            label = (sum(y)/len(y)) > 0.5

        return isleaf, label


    def _split_recurs(self, node, data, indices):
        '''
        TODO:
        Recursively split the node based on the rows and indices given.
        Nothing needs to be returned.

        First use _is_terminal() to check if the node needs to be split.
        If so, select the column that has the maximum infomation gain to split on.
        Store the label predicted for this node, the split column, and use _set_info()
        to keep track of the gain and the number of datapoints at the split.
        Then, split the data based on its value in the selected column.
        The data should be recursively passed to the children.
        '''
        # TODO: Check if node needs to be split
        node.isleaf, node.label = self._is_terminal(node, data, indices) 

        if not node.isleaf:
            # TODO: Initialize and Calculate Gain
            gain = np.zeros(len(indices))
            for i in range(len(indices)):
                gain[i] = self._calc_gain(data, indices[i], self.gain_function)

            # TODO: Split the column and use _set_info() 
            node._set_info(max(gain), len(data))
            node.index_split_on = indices[np.argmax(gain)]
            indices.remove(node.index_split_on)

            # TODO: Split the Data and Pass it recursively to the  children
            left_data, right_data = [], []
            for row in data:
                if not row[node.index_split_on]:
                    left_data.append(row)
                else:
                    right_data.append(row)

            node.left = Node(depth=node.depth+1)
            node.right = Node(depth=node.depth+1)

            self._split_recurs(node.left, left_data, copy.copy(indices))
            self._split_recurs(node.right, right_data, copy.copy(indices))
        

    def _calc_gain(self, data, split_index, gain_function):
        '''
        TODO:
        Calculate the gain of the proposed splitting and return it.
        Gain = C(P[y=1]) - P[x_i=True] * C(P[y=1|x_i=True]) - P[x_i=False] * C(P[y=0|x_i=False])
        Here the C(p) is the gain_function. For example, if C(p) = min(p, 1-p), this would be
        considering training error gain. Other alternatives are entropy and gini functions.
        '''
        y = [row[0] for row in data]
        xi = [row[split_index] for row in data]

        if len(y) != 0 and len(xi) != 0:
            # TODO: Calculate Gain
            pxi_T = sum(xi)/len(xi)
            if pxi_T != 0 and pxi_T != 1:
                gain = gain_function(sum(y)/len(y)) - \
                pxi_T * gain_function(sum((xi[j]==True) and (y[j]==1) for j in range(len(y)))/len(y) / pxi_T) - \
                (1-pxi_T) * gain_function(sum((xi[j]==False) and (y[j]==0) for j in range(len(y)))/len(y) / (1-pxi_T))
            else:
                gain = 0
        else:
            gain = 0
        return gain
    

    def print_tree(self):
        '''
        Helper function for tree_visualization.
        Only effective with very shallow trees.
        You do not need to modify this.
        '''
        print('---START PRINT TREE---')
        def print_subtree(node, indent=''):
            if node is None:
                return str("None")
            if node.isleaf:
                return str(node.label)
            else:
                decision = 'split attribute = {:d}; gain = {:f}; number of samples = {:d}'.format(node.index_split_on, node.info['gain'], node.info['num_samples'])
            left = indent + '0 -> '+ print_subtree(node.left, indent + '\t\t')
            right = indent + '1 -> '+ print_subtree(node.right, indent + '\t\t')
            return (decision + '\n' + left + '\n' + right)

        print(print_subtree(self.root))
        print('----END PRINT TREE---')


    def loss_plot_vec(self, data):
        '''
        Helper function to visualize the loss when the tree expands.
        You do not need to modify this.
        '''
        self._loss_plot_recurs(self.root, data, 0)
        loss_vec = []
        q = [self.root]
        num_correct = 0
        while len(q) > 0:
            node = q.pop(0)
            num_correct = num_correct + node.info['curr_num_correct']
            loss_vec.append(num_correct)
            if node.left != None:
                q.append(node.left)
            if node.right != None:
                q.append(node.right)

        return 1 - np.array(loss_vec)/len(data)


    def _loss_plot_recurs(self, node, rows, prev_num_correct):
        '''
        Helper function to visualize the loss when the tree expands.
        You do not need to modify this.
        '''
        labels = [row[0] for row in rows]
        curr_num_correct = labels.count(node.label) - prev_num_correct
        node.info['curr_num_correct'] = curr_num_correct

        if not node.isleaf:
            left_data, right_data = [], []
            left_num_correct, right_num_correct = 0, 0
            for row in rows:
                if not row[node.index_split_on]:
                    left_data.append(row)
                else:
                    right_data.append(row)

            left_labels = [row[0] for row in left_data]
            left_num_correct = left_labels.count(node.label)
            right_labels = [row[0] for row in right_data]
            right_num_correct = right_labels.count(node.label)

            if node.left != None:
                self._loss_plot_recurs(node.left, left_data, left_num_correct)
            if node.right != None:
                self._loss_plot_recurs(node.right, right_data, right_num_correct)


## **Check Model**

In [30]:
import pytest
np.random.seed(0)
random.seed(0)

# Tests for node_score_error
assert node_score_error(.3) == .3
assert node_score_error(.6) == .4

# Tests for node_score_entropy
assert node_score_entropy(.5) == pytest.approx(.69, .01)
assert node_score_entropy(0) == node_score_entropy(1) == 0
assert node_score_entropy(.7) == pytest.approx(.61,.01)

# Tests for node_score_gini
assert node_score_gini(1) == node_score_gini(0) == 0
assert node_score_gini(.4) == .48

### Finish TODO's in DecisionTree then run tests below

# Creates Test Model and Dummy Data
x = np.array([[0,1,0,0],[1,0,1,1],[1,1,0,1],[0,0,1,0],[0,1,1,1],[0,0,0,0]])
test_model = DecisionTree(x)

# Tests _calc_gain
assert test_model._calc_gain(x, 0, node_score_error) == pytest.approx(.333, .01)
assert test_model._calc_gain(x, 0, node_score_entropy) == pytest.approx(.64, .01)
assert test_model._calc_gain(x, 0, node_score_gini) == pytest.approx(.444, .01)

# Check Tree is created Properly, Compare with text below
test_model.print_tree()

# Tests _prune_recurs
# Pruned tree should be smaller
# with higher training loss and lower validation loss
x_val = np.array([[1,1,1,1],[1,0,0,1]])

print('training loss not pruned:', test_model.loss(x))
print('validation loss not pruned:', test_model.loss(x_val), '\n')

test_model_pruned = DecisionTree(x,validation_data=x_val)
test_model_pruned.print_tree()
print('training loss pruned:', test_model_pruned.loss(x))
print('validation loss pruned:', test_model_pruned.loss(x_val))

from datetime import date
#[TODO] Print your name with the date, using today function from date 

---START PRINT TREE---
split attribute = 3; gain = 0.318257; number of samples = 6
0 -> False
1 -> split attribute = 1; gain = 0.174416; number of samples = 3
		0 -> True
		1 -> split attribute = 2; gain = 0.693147; number of samples = 2
				0 -> True
				1 -> False
----END PRINT TREE---
training loss not pruned: 0.0
validation loss not pruned: 0.5 

---START PRINT TREE---
split attribute = 3; gain = 0.318257; number of samples = 6
0 -> False
1 -> True
----END PRINT TREE---
training loss pruned: 0.16666666666666666
validation loss pruned: 0.0


In [ ]:
'''
Decision Trees should look similar to below (the second one is the pruned tree)

---START PRINT TREE---
split attribute = 3; gain = 0.318257; number of samples = 6
0 -> False
1 -> split attribute = 1; gain = 0.174416; number of samples = 3
		0 -> True
		1 -> split attribute = 2; gain = 0.693147; number of samples = 2
				0 -> True
				1 -> False
----END PRINT TREE---


---START PRINT TREE---
split attribute = 3; gain = 0.318257; number of samples = 6
0 -> False
1 -> True
----END PRINT TREE---
training loss pruned: 0.16666666666666666
validation loss pruned: 0.0
'''

## **Main**

In [31]:
import matplotlib.pyplot as plt

def loss_plot(ax, title, tree, pruned_tree, train_data, test_data):
    '''
        Example plotting code. This plots four curves: the training and testing
        average loss using tree and pruned tree.
        You do not need to change this code!
        Arguments:
            - ax: A matplotlib Axes instance.
            - title: A title for the graph (string)
            - tree: An unpruned DecisionTree instance
            - pruned_tree: A pruned DecisionTree instance
            - train_data: Training dataset returned from get_data
            - test_data: Test dataset returned from get_data
    '''
    fontsize=8
    ax.plot(tree.loss_plot_vec(train_data), label='train non-pruned')
    ax.plot(tree.loss_plot_vec(test_data), label='test non-pruned')
    ax.plot(pruned_tree.loss_plot_vec(train_data), label='train pruned')
    ax.plot(pruned_tree.loss_plot_vec(test_data), label='test pruned')


    ax.locator_params(nbins=3)
    ax.set_xlabel('number of nodes', fontsize=fontsize)
    ax.set_ylabel('loss', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize)
    legend = ax.legend(loc='upper center', shadow=True, fontsize=fontsize-2)

def explore_dataset(filename, class_name):
    train_data, validation_data, test_data = get_data(filename, class_name)

    # TODO: Print 12 loss values associated with the dataset.
    # For each measure of gain (training error, entropy, gini):
    #      (a) Print average training loss (not-pruned)
    #      (b) Print average test loss (not-pruned)
    #      (c) Print average training loss (pruned)
    #      (d) Print average test loss (pruned)

    # TODO: Feel free to print or plot anything you like here. Just comment
    # make sure to comment it out, or put it in a function that isn't called
    # by default when you hand in your code!

    decision_entropy = DecisionTree(train_data)
    training_loss = decision_entropy.loss(train_data)
    print('entropy: training loss not pruned', training_loss)
    test_loss = decision_entropy.loss(test_data)
    print('entropy: test loss not pruned', test_loss)

    decision_entropy_pruned = DecisionTree(train_data,validation_data=validation_data)
    training_loss = decision_entropy_pruned.loss(train_data)
    print('entropy: training loss pruned', training_loss)
    test_loss = decision_entropy_pruned.loss(test_data)
    print('entropy: test loss pruned', test_loss)

    decision_gini = DecisionTree(train_data, gain_function=node_score_gini)
    training_loss = decision_gini.loss(train_data)
    print('gini: training loss not pruned', training_loss)
    test_loss = decision_gini.loss(test_data)
    print('gini: test loss not pruned', test_loss)

    decision_gini_pruned = DecisionTree(train_data,validation_data=validation_data, gain_function=node_score_gini)
    training_loss = decision_gini_pruned.loss(train_data)
    print('gini: training loss pruned', training_loss)
    test_loss = decision_gini_pruned.loss(test_data)
    print('gini: test loss pruned', test_loss)

    decision_error = DecisionTree(train_data, gain_function=node_score_error)
    training_loss = decision_error.loss(train_data)
    print('decision error: training loss not pruned', training_loss)
    test_loss = decision_error.loss(test_data)
    print('decision error: test loss not pruned', test_loss)

    decision_error_pruned = DecisionTree(train_data,validation_data=validation_data, gain_function=node_score_error)
    training_loss = decision_error_pruned.loss(train_data)
    print('decision error: training loss pruned', training_loss)
    test_loss = decision_error_pruned.loss(test_data)
    print('decision error: test loss pruned', test_loss)


########### PLEASE DO NOT CHANGE THESE LINES OF CODE! ###################
random.seed(1)
np.random.seed(1)
#########################################################################
explore_dataset('data/chess.csv', 'won')
explore_dataset('data/spam.csv', '1')


entropy: training loss not pruned 0.0
entropy: test loss not pruned 0.013377926421404682
entropy: training loss pruned 0.004
entropy: test loss pruned 0.01254180602006689
gini: training loss not pruned 0.0
gini: test loss not pruned 0.014214046822742474
gini: training loss pruned 0.004
gini: test loss pruned 0.013377926421404682
decision error: training loss not pruned 0.0
decision error: test loss not pruned 0.05100334448160535
decision error: training loss pruned 0.023
decision error: test loss pruned 0.04180602006688963
entropy: training loss not pruned 0.009
entropy: test loss not pruned 0.12956555171088044
entropy: training loss pruned 0.035
entropy: test loss pruned 0.1184159938485198
gini: training loss not pruned 0.009
gini: test loss not pruned 0.13341022683583237
gini: training loss pruned 0.044
gini: test loss pruned 0.11687812379853903
decision error: training loss not pruned 0.014
decision error: test loss not pruned 0.1318723567858516
decision error: training loss pruned 

## **Project Report**

### **Question 1**

Comment on the results of your final program. Discuss the
differences in training and test error of pruned and non-pruned
trees. Which measure of gain most effectively reduced training
error? Was pruning effective? Use tables or graphs to demonstrate
your findings.

**Example Solution:**

*Solutions should include: tables or graphs, discuss difference of training and testing error, discuss difference of pruning and not pruning, and discuss which measure of gain is most effective.*

In general, training error was lower than test error. This is to be expected, since the trees are built upon training data and thus fit the training data better. Pruning did appear to be effective, because the testing loss was lower for pruned trees than for unpruned trees. However, training error was in fact higher for pruned trees than for unpruned trees, because pruning fits the validation data more and thus the training data less.

The following table lists the training and testing loss for unpruned and pruned trees, for each of the 3 measures of loss: training error, entropy, and gini. The data used was the spam.csv dataset. We can see that both entropy and gini effectively reduced training error, with an average loss of both 0.014 as opposed to 0.016. After pruning, gini most effectively reduced training error.

<div>
<img src="hw6_report_01_solution.png" width="500"/>
</div>

### **Question 2**
Using the `spam.csv` dataset, plot the loss of your decision tree on
the *training* set for trees with maximum depth set to each value
between $1$ to $15$. For these plots, the trees should not be pruned
and you can use the entropy gain function. Discuss any trends you
find and attempt to explain them in three to five sentences.

**Example Solution:**

*Solutions should include: graph of loss, depths 1 to 15, and discussion about trends*

See the figure below. We plotted the loss of the decision tree on the training set with a maximum depth set to each value between 1 and 15, with the entropy gain function. The graph shows a decreasing trend. In particular, as depth increases the loss appears to decrease exponentially. As the depth increases by 1, there should be twice as many leaves as before rather than a linear increase in leaves. Hence, we have an exponential rather than linear graph.

![image](hw6_report_02_solution.png)
